In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import sys
# os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14982960516995668210
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5892931584
locality {
  bus_id: 1
}
incarnation: 5284471154733629196
physical_device_desc: "device: 0, name: GeForce GTX 980 Ti, pci bus id: 0000:02:00.0, compute capability: 5.2"
]


In [3]:
def compare_img(img1,img2):
    img1 = normalize_gray(img1)
    img2 = normalize_gray(img2)
    diff = img1 - img2
    m_norm = np.sum(abs(diff))
#     return m_norm
    return 0
def normalize_frame(frame):
    return frame/255

def normalize_gray(frame):
    rng = np.max(frame) - np.min(frame)
    amin = np.min(frame)
    return (frame-amin)/rng #range from [0,1]

In [4]:
from keras import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution3D, MaxPooling3D
from keras.utils import np_utils
from keras import optimizers

def gen_model():
    
    model = Sequential()
#     model.add(Dropout(0.3))
    model.add(Convolution3D(50,(2,5,5),strides=1,data_format='channels_last',input_shape = (33,96,128,5),activation='relu'))
#     model.add(Dropout(0.2))
#     model.add(Dropout(0.5))
    model.add(MaxPooling3D(pool_size=(2,2,2), padding='VALID'))
    model.add(Dropout(0.3))
    model.add(Convolution3D(100,(2,5,5),strides=1,data_format='channels_last',activation='relu'))
# #     model.add(Dropout(0.5))
    model.add(MaxPooling3D(pool_size=(2,2,2), padding='VALID'))
    model.add(Dropout(0.3))
    model.add(Convolution3D(200,(2,5,5),strides=1,data_format='channels_last',activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling3D(pool_size=(2,2,2), padding='VALID'))
# #     model.add(Convolution3D(70,(2,5,5),strides=1,activation='relu'))
# #     model.add(Dropout(0.5))
    model.add(Flatten())
#     model.add(Dense(2048, activation='relu'))
    model.add(Dense(128, activation='relu'))
#     model.add(Dense(512, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    print(model.summary())
    return model

model = gen_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 32, 92, 124, 50)   12550     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 16, 46, 62, 50)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 46, 62, 50)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 42, 58, 100)   250100    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 7, 21, 29, 100)    0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 7, 21, 29, 100)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 6, 17, 25, 200)    1000200   
__________

Using TensorFlow backend.


In [5]:
from keras.models import model_from_yaml

model.load_weights("adscnn/model-01-0.6914.h5")
print("Loaded model from disk")

# opti = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# model.compile(loss='categorical_crossentropy', optimizer=opti, metrics=['accuracy'])

Loaded model from disk


In [6]:
### HYPERPARAMETERS ###

RESIZE_FACTOR = 5
DILATION_FACTOR = 3
BUFFER_SIZE = 100
WIDTH = 640
HEIGHT = 480

#######################
definition = ['static', 'move', 'rot', 'frot', 'light', 'dynamiclight','unknown']
buffer = []
opticflow_buffer = []
cap = cv2.VideoCapture(0)
frame_count = 0

while(True):
    ret, frame = cap.read()
    frame = cv2.resize(frame, (0,0),
        fx=1/RESIZE_FACTOR, fy=1/RESIZE_FACTOR)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    opticflow_buffer.append(gray)
    if(len(opticflow_buffer) == 2):
        if(frame_count % DILATION_FACTOR == 0):
            hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
            hsv = np.subtract(np.multiply(np.divide(hsv,255),2),1)
            flow = cv2.calcOpticalFlowFarneback(opticflow_buffer[0],
                opticflow_buffer[1], None, 0.5, 3, 15, 3, 5, 1.2, 0)
            mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
            mag = cv2.normalize(mag,None,-1,1,cv2.NORM_MINMAX)
            ang = (ang*180/np.pi/2/180) - 1
            tmp = np.ndarray((HEIGHT//RESIZE_FACTOR,WIDTH//RESIZE_FACTOR,2))
            tmp[:,:,0] = mag
            tmp[:,:,1] = ang
            data_frame = np.ndarray((HEIGHT//RESIZE_FACTOR,WIDTH//RESIZE_FACTOR,5))
            data_frame[:, :, 0:3] = hsv
            data_frame[:, :, 3:5] = tmp
            buffer.append(data_frame)
            if(len(buffer) == BUFFER_SIZE//DILATION_FACTOR):
                x = np.asarray(buffer)
#                 print(x.shape)
                x = x.reshape(1,33,96,128,5)
                prediction = model.predict(x)
                sys.stdout.write("\r"+definition[np.argmax(prediction[0])])
                sys.stdout.flush()
                del buffer[0]
        cv2.imshow('frame',frame)
        frame_count += 1
        del opticflow_buffer[0]
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

static